1. TRATAMENTO DE DADOS POR DATA

In [ ]:
import pandas as pd

df = pd.read_csv('movies.csv', 
    parse_dates=['release_date'], 
    dayfirst=False, 
    na_values=['', 'NA', 'NaN'], 
    dtype={'id': 'Int64'})


# Extrai ano, mês e dia da semana da data de lançamento
df['release_year']        = df['release_date'].dt.year
df['release_month']       = df['release_date'].dt.month
df['release_day_of_week'] = df['release_date'].dt.dayofweek  # Segunda=0, Domingo=6

print("Visualização do DataFrame após o tratamento da data:")
print(df[['id', 'release_date', 'release_year', 'release_month', 'release_day_of_week']].head())
print(f"\nNúmero de datas inválidas ou faltantes: {df['release_date'].isnull().sum()}")

Visualização do DataFrame após o tratamento da data:
       id release_date  release_year  release_month  release_day_of_week
0  615656   2023-08-02        2023.0            8.0                  2.0
1  758323   2023-04-05        2023.0            4.0                  2.0
2  533535   2024-07-24        2024.0            7.0                  2.0
3  667538   2023-06-06        2023.0            6.0                  1.0
4  693134   2024-02-27        2024.0            2.0                  1.0

Número de datas inválidas ou faltantes: 51549


2. TRATAMENTO DE DADOS CATEGÓRICOS COM MÚLTIPLOS VALORES (genres)

In [ ]:
import pandas as pd

df = pd.read_csv('movies.csv')

# Tratamento n° 1
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year.astype('Int64')
df['release_month'] = df['release_date'].dt.month.astype('Int64')
df['release_day_of_week'] = df['release_date'].dt.dayofweek.astype('Int64')

# Tratamento n° 2
df['genres'] = (
    df['genres']
    .fillna('')  
    .str.replace(r'[\|,]', '-', regex=True)  
    .str.split('-')                          
    .apply(lambda lst: [g.strip() for g in lst if g.strip() != '']) 
    .str.join('-')                          
)

genres_dummies = df['genres'].str.get_dummies(sep='-')
genres_dummies.columns = [
    f"genre_{col.strip().replace(' ', '_')}" for col in genres_dummies.columns
]

# Agrupar gêneros raros em 'genre_Other'
threshold = 1000  # mínimo de ocorrências para manter como coluna própria
freq = genres_dummies.sum()
common = freq[freq >= threshold].index.tolist()
rare   = freq[freq <  threshold].index.tolist()

genres_dummies['genre_Other'] = genres_dummies[rare].sum(axis=1)
genres_dummies = genres_dummies[common + ['genre_Other']]

df = pd.concat([df, genres_dummies], axis=1)
df.drop(columns=['genres'], inplace=True)

print("Colunas de gênero após melhorias:")
print(genres_dummies.columns.tolist())
print("\nExemplo de linhas (ID + primeiras 9 colunas de gênero):")
print(df[['id'] + genres_dummies.columns.tolist()[:9]].head())


Colunas de gênero após melhorias:
['genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy', 'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family', 'genre_Fantasy', 'genre_History', 'genre_Horror', 'genre_Music', 'genre_Mystery', 'genre_Romance', 'genre_Science_Fiction', 'genre_TV_Movie', 'genre_Thriller', 'genre_War', 'genre_Western', 'genre_Other']

Exemplo de linhas (ID + primeiras 9 colunas de gênero):
       id  genre_Action  genre_Adventure  genre_Animation  genre_Comedy  \
0  615656             1                0                0             0   
1  758323             0                0                0             0   
2  533535             1                0                0             1   
3  667538             1                1                0             0   
4  693134             0                1                0             0   

   genre_Crime  genre_Documentary  genre_Drama  genre_Family  genre_Fantasy  
0            0                  0    

3. TRATAMENTO DE DADOS NUMÉRICOS ('budget', 'revenue')

In [ ]:
import pandas as pd

df = pd.read_csv('movies.csv')

# Tratamento n° 1
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month
df['release_day_of_week'] = df['release_date'].dt.dayofweek

# Tratamento n° 2
df['genres'] = df['genres'].fillna('')
genres_dummies = df['genres'].str.get_dummies(sep='-').add_prefix('genre_')
df = pd.concat([df, genres_dummies], axis=1)
df = df.drop('genres', axis=1)

# Tratamento n° 3 
linhas_antes_da_limpeza = len(df)
# Manter apenas as linhas onde o orçamento E a arrecadação são maiores que zero.
df = df[(df['budget'] > 0) & (df['revenue'] > 0)]

print(f"Linhas antes da limpeza de orçamento/arrecadação: {linhas_antes_da_limpeza}")
print(f"Linhas após a limpeza (dataset final): {len(df)}")
print(f"Total de {linhas_antes_da_limpeza - len(df)} linhas foram removidas.")
print(df[['id', 'budget', 'revenue', 'release_year', 'genre_Action']].head()) # genre_Action é um exemplo de gênero tratado

Linhas antes da limpeza de orçamento/arrecadação: 722317
Linhas após a limpeza (dataset final): 11277
Total de 711040 linhas foram removidas.
       id       budget       revenue  release_year  genre_Action
0  615656  129000000.0  3.520565e+08        2023.0             1
1  758323   18000000.0  6.567582e+07        2023.0             0
2  533535  200000000.0  1.326387e+09        2024.0             1
3  667538  200000000.0  4.070455e+08        2023.0             1
4  693134  190000000.0  6.838137e+08        2024.0             0
